#검증 세트(Validation set)
---
지금까지 모델의 성능을 평가할 때 훈련 세트에서 모델을 훈련하고 테스트 세트에서 모델을 평가했다. 하지만 테스트 세트를 사용하여 성능을 확인하다보면 점점 테스트 세트에 맞추게 된다. 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 테스트 세트를 사용하지 않고 모델을 만든 후 마지막에 딱 한번만 사용하는 것이 좋다. 테스트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. 이를 위해 훈련 세트를 다시 나누어 **검증 세트**를 만든다. 보통 훈련 세트 약 60%, 검증 세트 약 20%, 테스트 세트 약 20%로 나눈다.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42) # 검증 세트

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target), dt.score(val_input, val_target)) # 과대 적합

0.9971133028626413 0.864423076923077


#교차 검증(Cross Validation)
---
검증 세트를 만들게 되면, 그만큼 훈련 세트가 줄어든다. 많은 데이터를 사용할수록 좋은 모델이 만들어지므로 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한 후 이 점수를 평균하여 최종 검증 점수를 얻는 **교차 검증**을 이용하자.


In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00827098, 0.00720501, 0.00717831, 0.00691056, 0.006742  ]), 'score_time': array([0.00091243, 0.00075889, 0.00069714, 0.00073767, 0.00071096]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score'])) # 검증 폴드의 점수

0.855300214703487


앞서 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 만들었기 때문에 따로 섞을 필요가 없었지만, 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 한다. 사이킷런의 분할기는 폴드를 어떻게 나눌지 결정해 준다. cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold 분할기를 사용한다.

In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42) # 10 폴드 교차 검증을 수행할 때
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


#하이퍼파라미터 튜닝
---
머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 하고, 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 한다.
여러 하이퍼파라미터가 존재하는 모델에서 한 매개변수의 최적값을 찾은 후 이를 고정하고 다른 매개변수의 최적값을 찾아도 될까? 안된다. 여러 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 한다. 이를 위해 **그리드 서치(Grid Search)**를 사용하자.

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # min_impurity_decrease의 값마다 5-폴드 교차 검증 수행
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1) # 그리드 서치 객체 / n_jobs는 매개변수에서 병렬 실행에 사용할 CPU 코어 수(-1은 시스템에 있는 모든 코어를 사용하겠다는 뜻)
gs.fit(train_input, train_target)

dt = gs.best_estimator_ # 그리드 서치는 훈련이 끝나면 25개의 모델 중 검증 점수가 가장 높은 모델의 매개변수 조합으로 자동으로 모델을 훈련해준다.
print('최적의 매개변수:', gs.best_params_)
print('각 매개변수에서 수행한 교차 검증의 평균 점수:', gs.cv_results_['mean_test_score'])
print(dt.score(train_input, train_target))

최적의 매개변수: {'min_impurity_decrease': 0.0001}
각 매개변수에서 수행한 교차 검증의 평균 점수: [0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
0.9615162593804117


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print('argmax()로 찾은 최적의 매개변수:', gs.cv_results_['params'][best_index])

argmax()로 찾은 최적의 매개변수: {'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)} # 매개변수로 수행할 교차 검증 횟수는 9 x 15 x 10 = 1,350에 5-폴드이므로 6,750

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)
print(gs.best_params_) # 최상의 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 점수

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


#랜덤 서치(Random Search)
---
매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또한 매개변수 조건이 너무 많은 경우 그리드 서치 수행 시간이 오래 걸릴 수 있다. 이럴 때 랜덤 서치를 사용하면 좋다. 랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.


In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([5, 2, 5, 9, 5, 1, 8, 8, 1, 5])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True) # 1000개를 샘플링해서 각 숫자의 개수를 살펴보자.

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([112, 102, 122, 108,  78,  85, 109,  92,  93,  99]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.36006394, 0.05906056, 0.0791981 , 0.76071244, 0.84847389,
       0.47287965, 0.98567939, 0.01918763, 0.54878232, 0.50545499])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease': np.arange(0.0001, 0.001),
          'max_depth': range(20, 50),
          'min_samples_split': range(2, 25),
          'min_samples_leaf': randint(1, 25)} # 탐색할 매개변수의 딕셔너리, min_samples_leaf는 리프 노드가 되기 위한 최소 샘플 개수

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 31, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 23, 'min_samples_split': 21}
0.8683852817057822
0.8576923076923076
